In [ ]:
#r "nuget: Newtonsoft.Json, 13.0.1"
#r "nuget: QuikGraph, 2.3.0"
#r "nuget: Plotly.NET, 2.0.0-preview.17"
#r "nuget: Plotly.NET.Interactive, 2.0.0-preview.17"

Installed Packages Newtonsoft.Json, 13.0.1 Plotly.NET, 2.0.0-preview.17 Plotly.NET.Interactive, 2.0.0-preview.17 QuikGraph, 2.3.0

Loading extensions from `Plotly.NET.Interactive.dll`

In [ ]:
#r "/Users/jnmatthews/MGGG/GerryChain.NET/GerryChain/bin/Release/net6.0/GerryChain.dll"
#r "/Users/jnmatthews/MGGG/GerryChain.NET/GerryChainExtensions/PCompress/bin/Release/net6.0/PCompress.dll"

In [ ]:
using System;
using System.Linq;
using Newtonsoft.Json;
using Newtonsoft.Json.Linq;
using Plotly.NET;
using Plotly.NET.Interactive;
using Plotly.NET.LayoutObjects;
using GerryChain;
using PCompress;

In [ ]:
var TallyBVAP = Scores.TallyFactory("BVAP", "BVAP");
var TallyVAP = Scores.TallyFactory("VAP", "VAP");
var TallyPOP = Scores.TallyFactory("TOTPOP", "TOTPOP");
var initPart = new Partition("../resources/al_vtds20_with_seeds.json", "CD_Seed", "TOTPOP", new string[] { "TOTPOP", "VAP", "BVAP" },
                             new Score[] { TallyBVAP, TallyPOP, TallyVAP }, regionAware: true, regionDivisionSpecs: new (string, double)[]{("COUNTY", 1)});
var chain = new Chain(initPart, 1000, 0.05, batchSize:4);

In [ ]:
var bs = chain.Select(p => ((DistrictWideScoreValue) p.Score("BVAP")).Value.Zip(((DistrictWideScoreValue)p.Score("VAP")).Value, (b, v) => b/v)).ToArray();

In [ ]:
bs[0]

index,value
0,0.244974735891391
1,0.16951728221183604
2,0.18607587146687765
3,0.13258485970646555
4,0.3611319531617938
5,0.36776413657137735
6,0.28313392857142855


## Recording a chain

In [ ]:
var rec = new Recorder(chain, "test.chain");
rec.Record();

## Replaying the previous chain

In [ ]:
var g = GraphParsers.LoadGraphFromJson("../resources/al_vtds20_with_seeds.json", "TOTPOP", new string[] { "TOTPOP", "VAP", "BVAP" });

In [ ]:
var replayer = new Replayer(g.graph, new Score[] { TallyBVAP, TallyPOP, TallyVAP }, "test.chain");

In [ ]:
var dist1BVAPs = new double[1000];
var numCutEdges = new int[1000];
int i = 0;

foreach (Partition p in replayer.Replay())
{
    numCutEdges[i] = p.CutEdges.Count();
    dist1BVAPs[i] = ((DistrictWideScoreValue)p.Score("BVAP")).Value [0];
    i++;
}

In [ ]:
GenericChart.GenericChart chart = Chart2D.Chart.Histogram<int,int,int>(X: numCutEdges)
    .WithYAxisStyle(title: Title.init("CutEdges"), ShowGrid: false, ShowLine: true);

In [ ]:
numCutEdges

index,value
0,772
1,774
2,812
3,790
4,766
5,762
6,772
7,774
8,792
9,704
